# Get Available Contracts with respective strike prices
## Chosen stock - BANKNIFTY

In [51]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import time
import string
import shutil
import os

In [50]:
## Configurations
download_location = r"/Users/mayank.gupta/Moneygen/Downloads"
data_file_location = r"/Users/mayank.gupta/Moneygen/DataFiles"
sleep_duration = 3
options = webdriver.ChromeOptions() 
options.add_experimental_option("prefs", {
  "download.default_directory": download_location,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

### Load the chrome webdriver

In [3]:
driver = webdriver.Chrome(executable_path = './chromedriver', chrome_options = options)

/Users/mayank.gupta/venv/tradeai/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


### Get the web page using driver for BANKNIFTY

In [4]:
driver.get("https://nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuoteFO.jsp?underlying=BANKNIFTY&instrument=FUTIDX&type=-&strike=-&expiry=25JUL2019")
time.sleep(sleep_duration)

### Select 'Options' as instrument type

In [5]:
instrument_type_element = driver.find_element_by_id("instruments")
instrument_selector = Select(instrument_type_element)
## Index is always index + 1 because 0 index corresponds to 'Select...'
instrument_selector.select_by_index(2)
time.sleep(sleep_duration)

### Get expiry dates of all available contracts

In [6]:
expiry_dates_element = driver.find_element_by_id("expiryDates")
expiry_dates = expiry_dates_element.text.split('\n')[2:].copy()
time.sleep(sleep_duration)

In [7]:
expiry_dates

['25JUL2019',
 '18JUL2019',
 '11JUL2019',
 '04JUL2019',
 '01AUG2019',
 '08AUG2019',
 '29AUG2019',
 '14AUG2019',
 '22AUG2019',
 '26SEP2019']

### Select expiry dates and populate the strike price list

In [8]:
date_price_dict = dict()

expiry_dates_selector = Select(expiry_dates_element)
for index in range(len(expiry_dates)):
    print('Getting strike prices for: ', expiry_dates[index], ', having index: ', index)
    indexer = index + 1
    date_price_dict[expiry_dates[index]] = dict()
    
    ## Select expiry for a contract
    expiry_dates_selector.select_by_index(indexer)
    time.sleep(sleep_duration)
    
    ## Selecting 'Call' option to populate 'Strike Price' list
    Select(driver.find_element_by_id("optionType")).select_by_index(1)
    time.sleep(sleep_duration)
    
    ## Populating strike prices for Call option of selected date
    strike_price_element = driver.find_element_by_id('strikePrices')
    date_price_dict[expiry_dates[index]]['Call'] = strike_price_element.text.split('\n')[2:].copy()
    print('Length of strike price list for ', expiry_dates[index], 'Call option: ', len(date_price_dict[expiry_dates[index]]['Call']))
    time.sleep(sleep_duration)
    
    ## Selecting 'Put' option to populate 'Strike Price' list
    Select(driver.find_element_by_id("optionType")).select_by_index(2)
    time.sleep(sleep_duration)
    
    ## Populating strike prices for Put option of selected date
    strike_price_element = driver.find_element_by_id('strikePrices')
    date_price_dict[expiry_dates[index]]['Put'] = strike_price_element.text.split('\n')[2:].copy()
    print('Length of strike price list for ', expiry_dates[index], 'Put option: ', len(date_price_dict[expiry_dates[index]]['Put']))
    time.sleep(sleep_duration)

Getting strike prices for:  25JUL2019 , having index:  0
Length of strike price list for  25JUL2019 Call option:  64
Length of strike price list for  25JUL2019 Put option:  64
Getting strike prices for:  18JUL2019 , having index:  1
Length of strike price list for  18JUL2019 Call option:  61
Length of strike price list for  18JUL2019 Put option:  49
Getting strike prices for:  11JUL2019 , having index:  2
Length of strike price list for  11JUL2019 Call option:  64
Length of strike price list for  11JUL2019 Put option:  50
Getting strike prices for:  04JUL2019 , having index:  3
Length of strike price list for  04JUL2019 Call option:  64
Length of strike price list for  04JUL2019 Put option:  64
Getting strike prices for:  01AUG2019 , having index:  4
Length of strike price list for  01AUG2019 Call option:  36
Length of strike price list for  01AUG2019 Put option:  22
Getting strike prices for:  08AUG2019 , having index:  5
Length of strike price list for  08AUG2019 Call option:  40
Len

In [9]:
month_mapper = {
"JAN":"-01-",
"FEB":"-02-",
"MAR":"-03-",
"APR":"-04-",
"MAY":"-05-",
"JUN":"-06-",
"JUL":"-07-",
"AUG":"-08-",
"SEP":"-09-",
"OCT":"-10-",
"NOV":"-11-",
"DEC":"-12-"
}

In [10]:
expiry_dates_mapper = dict()
## Change month name to month number
for index, date in enumerate(expiry_dates):
    month_name = expiry_dates[index][2:5]
    month_number = month_mapper[month_name]
    expiry_dates_mapper[expiry_dates[index]] = expiry_dates[index].replace(month_name, month_number)

In [11]:
expiry_dates_mapper

{'25JUL2019': '25-07-2019',
 '18JUL2019': '18-07-2019',
 '11JUL2019': '11-07-2019',
 '04JUL2019': '04-07-2019',
 '01AUG2019': '01-08-2019',
 '08AUG2019': '08-08-2019',
 '29AUG2019': '29-08-2019',
 '14AUG2019': '14-08-2019',
 '22AUG2019': '22-08-2019',
 '26SEP2019': '26-09-2019'}

In [12]:
for old_key, new_key in zip(expiry_dates_mapper.keys(), expiry_dates_mapper.values()):
    date_price_dict[new_key] = date_price_dict.pop(old_key)

In [13]:
date_price_dict.keys()

dict_keys(['25-07-2019', '18-07-2019', '11-07-2019', '04-07-2019', '01-08-2019', '08-08-2019', '29-08-2019', '14-08-2019', '22-08-2019', '26-09-2019'])

# Get historical data for All contracts -> All Strike Prices

In [14]:
## Configurations
instrument_type = 'Index Options'
symbol = 'BANK NIFTY'
year = '2019'

In [69]:
driver = webdriver.Chrome(executable_path = './chromedriver', chrome_options = options)
driver.get('https://www.nseindia.com/products/content/derivatives/equities/historical_fo.htm')

/Users/mayank.gupta/venv/tradeai/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [70]:
Select(driver.find_element_by_id('instrumentType')).select_by_index(3) ## Select `Index Options`
time.sleep(sleep_duration)
Select(driver.find_element_by_id('symbol')).select_by_index(4) ## Select `BANK NIFTY`
time.sleep(sleep_duration)
Select(driver.find_element_by_id('dateRange')).select_by_index(6) ## Select `BANK NIFTY`
time.sleep(sleep_duration)
Select(driver.find_element_by_id('year')).select_by_index(5) ## Select `BANK NIFTY`
time.sleep(sleep_duration)

In [71]:
expiry_date_element = driver.find_element_by_id('expiryDate') ## Get expiryDates on website
expiry_date_list = expiry_date_element.text.replace(' ','').split('\n') ## Remove white spaces and split by newline

In [72]:
## For Call options
for index, item in enumerate(expiry_date_list):
    print(item)
    if item in list(expiry_dates_mapper.values()):
#         print('Inside if')
        Select(expiry_date_element).select_by_index(index)
        Select(driver.find_element_by_id("optionType")).select_by_index(1)
        time.sleep(sleep_duration)
        for strike_price in date_price_dict[item]['Call']:
            driver.find_element_by_id('strikePrice').clear()
            driver.find_element_by_id('strikePrice').send_keys(int(strike_price.split('.')[0]))
            driver.find_element_by_id('getButton').click()
            time.sleep(sleep_duration)
            try: 
                driver.find_element_by_link_text('Download file in csv format').click()
            except:
                print('Exception occured, waiting and trying again')
                driver.find_element_by_id('getButton').click()
                time.sleep(sleep_duration + 10)
                driver.find_element_by_link_text('Download file in csv format').click()
            time.sleep(sleep_duration)
            filename = os.listdir(download_location)[0]
            destination_filename = item + '_' + 'Call' + '_' + strike_price.split('.')[0] + '.csv'
            shutil.move(os.path.join(download_location,filename),os.path.join(data_file_location, destination_filename))
        

SelectExpiry
03-01-2019
10-01-2019
17-01-2019
24-01-2019
31-01-2019
07-02-2019
14-02-2019
21-02-2019
28-02-2019
07-03-2019
14-03-2019
15-03-2019
20-03-2019
28-03-2019
04-04-2019
11-04-2019
18-04-2019
25-04-2019
02-05-2019
09-05-2019
16-05-2019
23-05-2019
30-05-2019
06-06-2019
13-06-2019
20-06-2019
21-06-2019
27-06-2019
04-07-2019
Exception occured, waiting and trying again
Exception occured, waiting and trying again
Exception occured, waiting and trying again


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"Download file in csv format"}
  (Session info: chrome=75.0.3770.100)
